In [1]:
import requests
import json
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import os
import webbrowser
import folium
from folium import plugins
print(folium.__version__)

0.14.0


In [2]:
m = folium.Map(location=[36, 127], tiles="OpenStreetMap", zoom_start=7)

In [3]:
# folium.CircleMarker([37.5072384, 127.0238756],
#   radius=100,
#   color='blue',
#   fill_color='skyblue').add_to(m)

In [4]:
state_geo = 'sig_WGS84.json'

In [5]:
state_geo

'sig_WGS84.json'

In [6]:
state_data = pd.read_csv('Total_People_2023_2.csv', encoding = 'euc-kr')


In [7]:
import json

with open('sig_WGS84.json', 'r', encoding='utf-8') as f:

    json_data = json.load(f)


In [8]:
import json
json_data=open(state_geo, encoding='utf-8').read()
jsonResult = json.loads(json_data)

In [9]:
# 중앙위치 계산
def center_calc(points_df):
    x = points_df.x
    y = points_df.y

    X = (max(x)+min(x))/2.
    Y = (max(y)+min(y))/2.

    return X, Y

In [10]:
# 다중 Array 구조 이중으로 변환
def points_array(points):

    final_points = []

    for x in range(0, len(points)):

        if len(points[x]) == 2:
            final_points.append(points[x])
        else:
            target = points[x]
            for y in range(0, len(target)):
                final_points.append(target[y])

    return final_points

In [11]:
center_locations = pd.DataFrame()
codes = []
names = []
x_list = []
y_list = []
for x in range(0, len(jsonResult['features'])):
    code = jsonResult['features'][x]['properties']['SIG_CD']
    name = jsonResult['features'][x]['properties']['SIG_KOR_NM']
    # 중앙값 생성
    points = jsonResult['features'][x]['geometry']['coordinates'][0]
    points = points_array(points)
    points_df = pd.DataFrame(points)
    points_df.columns = ['x','y']
    X, Y = center_calc(points_df)

    # 결과
    codes.append(code)
    names.append(name)
    x_list.append(X)
    y_list.append(Y)

# 데이터 프레임 생성
center_locations['CODE'] = codes
center_locations['NAME'] = names
center_locations['X'] = x_list
center_locations['Y'] = y_list

In [12]:
state_data.CODE.dtype

dtype('int64')

In [13]:
center_locations.CODE.dtype

dtype('O')

In [14]:
center_locations.CODE.dtype == state_data.CODE.dtype

False

In [15]:
import numpy as np

In [16]:
state_data.CODE = state_data.CODE.astype(str) 
#state_data.CODE.dtype

In [17]:
target_df = pd.merge(state_data,center_locations,on='CODE')
target_df

,CODE,POPULATION,NAME,X,Y
0,11110,140032,종로구,126.9860,37.5990
1,11140,121482,중구,126.9945,37.5580
2,11170,216547,용산구,126.9815,37.5315
3,11200,278750,성동구,127.0415,37.5505
4,11215,336446,광진구,127.0855,37.5485
...,...,...,...,...,...
245,48870,37117,함양군,127.7315,35.5435
246,48880,60184,거창군,127.8870,35.7120
247,48890,41531,합천군,128.1670,35.6000
248,50110,492379,제주시,126.2270,33.4085


In [18]:
target_df = target_df[~np.isnan(target_df['X'])] # 위치 정보 없는 값 제외

In [19]:
# Initialize the map:
m = folium.Map(location=[36, 127], tiles="OpenStreetMap", zoom_start=7)


# I can add marker one by one on the map
for i in range(0,len(target_df)):
    latitude = target_df.iloc[i]['Y']
    longitude = target_df.iloc[i]['X']
    location=(latitude, longitude)
    folium.CircleMarker(location, radius=target_df.iloc[i]['POPULATION']/25000,color='green',fill_color='green', popup=target_df.iloc[i]['NAME']).add_to(m)
##location : 동그라미를 표시할 중심좌표 / radius : 동그라미 크기 / popup : 장소에 대한 세부정보 


folium.LayerControl(collapsed=False).add_to(m)



In [20]:
# Save to html
m.save(os.path.join('kr_polulation.html'))
m
# webbrowser.open_new("kr_polulation.html")